<img width="800px" src="../fidle/img/header.svg"></img>

# <!-- TITLE --> [K3LADYB1] - Prediction of a 2D trajectory via RNN
<!-- DESC --> Artificial dataset generation and prediction attempt via a recurrent network, using Keras 3 and PyTorch
<!-- AUTHOR : Jean-Luc Parouty (CNRS/SIMaP) -->

## Objectives :
 - Understanding the use of a recurrent neural network

## What we're going to do :

 - Generate an artificial dataset
 - dataset preparation
 - Doing our testing
 - Making predictions

## Step 1 - Import and init
### 1.1 - Python

In [1]:
import os
os.environ['KERAS_BACKEND'] = 'torch'

import keras
import numpy as np
from math import cos, sin
import random

import fidle

# Init Fidle environment
run_id, run_dir, datasets_dir = fidle.init('K3LADYB1')

<br>**FIDLE - Environment initialization**

Version              : 2.3.2
Run id               : K3LADYB1
Run dir              : ./run/K3LADYB1
Datasets dir         : /data/datasets-fidle
Start time           : 10/07/24 16:07:23
Hostname             : ebf2030c9b9e (Linux)
Tensorflow log level : Info + Warning + Error  (=0)
Update keras cache   : False
Update torch cache   : False
Save figs            : ./run/K3LADYB1/figs (False)
keras                : 3.0.5
numpy                : 1.26.4
sklearn              : 1.4.1.post1
yaml                 : 6.0.1
matplotlib           : 3.8.3
pandas               : 2.2.1
torch                : 2.2.1+cpu


### 1.2 - Parameters

In [2]:
# ---- About dataset
#
max_t        = 1000
delta_t      = 0.01
features_len = 2


sequence_len = 20
predict_len  = 5

# ---- About training
#
scale         = .2       # Percentage of dataset to be used (1=all)
train_prop    = .8       # Percentage for train (the rest being for the test)
batch_size    = 32
epochs        = 5
fit_verbosity = 1        # 0 = silent, 1 = progress bar, 2 = one line per epoch

Override parameters (batch mode) - Just forget this cell

In [3]:
fidle.override('scale', 'train_prop', 'sequence_len', 'predict_len', 'batch_size', 'epochs')

## Step 2 - Generation of a fun dataset
### 2.1 - Virtual trajectory of our ladybug

In [8]:
def ladybug_init(s=122):
    
    if s>0 : random.seed(s)
    ladybug_init.params_x = [ random.gauss(0.,1.) for u in range(8)]
    ladybug_init.params_y = [ random.gauss(0.,1.) for u in range(8)]
    
def ladybug_move(t):

    [ax1, ax2, ax3, ax4, kx1, kx2, kx3, kx4] = ladybug_init.params_x
    [ay1, ay2, ay3, ay4, ky1, ky2, ky3, ky4] = ladybug_init.params_y
    
    x = ax1*sin(t*(kx1+20)) + ax2*cos(t*(kx2+10)) + ax3*sin(t*(kx3+5)) + ax4*cos(t*(kx4+5))
    y = ay1*cos(t*(ky1+20)) + ay2*sin(t*(ky2+10)) + ay3*cos(t*(ky3+5)) + ay4*sin(t*(ky4+5)) 

    return x,y

<class 'function'>


### 2.2 - Get some positions, and build a rescaled and normalized dataset

In [ ]:
# ---- Get positions
#
ladybug_init(s=16)
x,y = 0,0
positions=[]
for t in np.arange(0., max_t, delta_t):
    x,y = ladybug_move(t)
    positions.append([x,y])

# ---- Build rescaled dataset
#
n = int( len(positions)*scale )
dataset = np.array(positions[:n])

k = int(len(dataset)*train_prop)
x_train = dataset[:k]
x_test  = dataset[k:]

# ---- Normalize
#
mean = x_train.mean()
std  = x_train.std()
x_train = (x_train - mean) / std
x_test  = (x_test  - mean) / std

print("Dataset generated.")
print("Train shape is : ", x_train.shape)
print("Test  shape is : ", x_test.shape)


### 2.3 - Have a look
An extract from the data we have: the virtual trajectory of our ladybug   
And what we want to predict (in red), from a segment (in blue)

In [ ]:
fidle.scrawler.serie_2d(x_train[:1000], figsize=(12,12), lw=1,ms=4,save_as='01-dataset')

In [ ]:
k1,k2 = sequence_len, predict_len
i = random.randint(0,len(x_test)-k1-k2)
j = i+k1

fidle.scrawler.segment_2d( x_test[i:j+k2], x_test[j:j+k2],ms=6, save_as='02-objectives')

### 2.4 - Prepare sequences from datasets

In [ ]:
# ---- Create sequences and labels for train and test
#
xs_train, ys_train=[],[]
all_i = np.random.permutation( len(x_train) - sequence_len - 1 )

for i in all_i:
    xs_train.append( x_train[ i : i+sequence_len ] )
    ys_train.append( x_train[ i+sequence_len+1 ]   )
    
xs_test, ys_test=[],[]
for i in range( len(x_test) - sequence_len - 1):
    xs_test.append( x_test[ i : i+sequence_len ] )
    ys_test.append( x_test[ i+sequence_len+1 ]   )

# ---- Convert to numpy / float16
    
xs_train = np.array(xs_train, dtype='float16')
ys_train = np.array(ys_train, dtype='float16')
xs_test  = np.array(xs_test,  dtype='float16')
ys_test  = np.array(ys_test,  dtype='float16')


In [ ]:
fidle.utils.subtitle('About the splitting of our dataset :')

print('Number of sequences : ', len(xs_train))
print('xs_train shape      : ',xs_train.shape)
print('ys_train shape      : ',ys_train.shape)

fidle.utils.subtitle('What an xs look like :')
fidle.utils.np_print(xs_train[10] )
fidle.utils.subtitle('What an ys look like :')
fidle.utils.np_print(ys_train[10])

## Step 3 - Create a model

In [ ]:
model = keras.models.Sequential()
model.add( keras.layers.InputLayer(shape=(sequence_len, features_len)) )
model.add( keras.layers.GRU(200, return_sequences=False, activation='relu') )
model.add( keras.layers.Dense(features_len) )

model.summary()

model.compile(optimizer='rmsprop', 
              loss='mse', 
              metrics   = ['mae'] )

## Step 4 - Train the model
### 4.1 Add Callbacks

In [ ]:
os.makedirs(f'{run_dir}/models',   mode=0o750, exist_ok=True)
save_dir = f'{run_dir}/models/best_model.keras'

savemodel_callback = keras.callbacks.ModelCheckpoint( filepath=save_dir, monitor='val_mae', mode='max', save_best_only=True)

### 4.2 - Train it
Need 3' on a cpu laptop

In [ ]:
chrono=fidle.Chrono()
chrono.start()

history=model.fit(xs_train,ys_train,
                  epochs  = epochs, 
                  verbose = fit_verbosity,
                  validation_data = (xs_test, ys_test),
                  callbacks = [savemodel_callback])

chrono.show()

In [ ]:
fidle.scrawler.history(history,plot={'loss':['loss','val_loss'], 'mae':['mae','val_mae']}, save_as='03-history')

## Step 5 - Predict

### 5.1 - Load model

In [ ]:
loaded_model = keras.models.load_model(f'{run_dir}/models/best_model.keras')
print('Loaded.')

### 5.2 - Make a 1-step prediction
A simple prediction on a single iteration

In [ ]:
s=random.randint(0,len(x_test)-sequence_len)

sequence      = x_test[s:s+sequence_len]
sequence_true = x_test[s:s+sequence_len+1]

sequence_pred = loaded_model.predict( np.array([sequence]), verbose=fit_verbosity )

print('sequence shape      :',sequence.shape)
print('sequence true shape :',sequence_true.shape)
print('sequence pred shape :',sequence_pred.shape)

fidle.scrawler.segment_2d(sequence_true, sequence_pred, save_as='04-one-step-prediction')
fidle.scrawler.multivariate_serie(sequence_true, predictions=sequence_pred, labels=['Axis=0', 'Axis=1'],save_as='05-one-step-prediction-2axis')

### 5.3 - Make n-steps prediction
A longer term prediction, via a nice iteration function  
We will perform <iteration> predictions to iteratively build our prediction.

In [ ]:
def get_prediction(dataset, model, iterations=4):

    # ---- Initial sequence
    #
    s=random.randint(0,len(dataset)-sequence_len-iterations)

    sequence_pred = dataset[s:s+sequence_len].copy()
    sequence_true = dataset[s:s+sequence_len+iterations].copy()

    # ---- Iterate 
    #
    sequence_pred = list(sequence_pred)

    for i in range(iterations):
        sequence   = sequence_pred[-sequence_len:]
        prediction = model.predict( np.array([sequence]), verbose=fit_verbosity )
        sequence_pred.append(prediction[0])

    # ---- Extract the predictions    
    #
    prediction = np.array(sequence_pred[-iterations:])

    return sequence_true,prediction

An n-steps prediction :

In [ ]:
sequence_true, sequence_pred = get_prediction(x_test, loaded_model, iterations=5)

fidle.scrawler.segment_2d(sequence_true, sequence_pred, ms=8, save_as='06-n-steps-prediction-norm')
fidle.scrawler.multivariate_serie(sequence_true, predictions=sequence_pred, hide_ticks=True, labels=['Axis=0', 'Axis=1'],save_as='07-n-steps-prediction-norm')

In [ ]:
fidle.end()

---
<img width="80px" src="../fidle/img/logo-paysage.svg"></img>